In [18]:
import numpy as np
import FunctionCollection as fc
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [14]:
args = {'eps': 0}

crit, yf, of, cal_acc = fc.Loss_Functions('Spherical_NLLH',args)

Remember all accuracies are positive and defined to go towards 0 in the optimal case.


In [40]:
def cos_diff_angle(azp,zep,azt,zet):
    from numpy import sin, cos, abs
    term1 = abs(sin(zep))*cos(azp)*sin(zet)*cos(azt)
    term2 = abs(sin(zep))*sin(azp)*sin(zet)*sin(azt)
    term3 = cos(zep)*cos(zet)
    return term1 + term2 + term3

In [46]:
pred = np.array([[2*0.5,0.52]])*np.pi
label = np.array([[2*0.5,0.5]])*np.pi

cos_diff_angle(pred[:,0],pred[:,1],label[:,0],label[:,1])

array([0.99802673])

In [24]:
cal_acc(torch.tensor(pred),torch.tensor(label))

TypeError: cos_diff_angle() missing 2 required positional arguments: 'zep' and 'zet'